In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
df = pd.read_csv('../development/K562_attributions.csv')
fig = go.Figure()
# fig.add_trace(go.Scatter(df))
# fig.add_trace(go.Scatter(x=list(df.index[df['chrom'] == 'chr1']*2000),
marks = ['H2A.Z', 'H3K27ac', 'H3K79me2', 'H3K27me3',
                                 'H3K9ac', 'H3K4me2', 'H3K4me3', 'H3K9me3',
                                 'H3K4me1', 'H3K36me3', 'H4K20me1']
fig = px.line(df, x=df.index*2000, y=df.columns[1:])
fig.update_layout(
    title="Attributions from integrated of gradients",
    xaxis_title="Genomic position (b)",
    yaxis_title="attributions and podls",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
fig.show()
fig.write_html("../development/attributions_profile.html")

SyntaxError: invalid syntax (1166596459.py, line 16)